In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

def download_stock_data(symbol, start='2020-01-01', end='2024-01-01'):
    stock = yf.download(symbol, start=start, end=end)
    return stock
def stock_add_indicator(df, sma, lma):
    df['SMA'] = df.Close.rolling(window=sma).mean()
    df['LMA'] = df.Close.rolling(window=lma).mean()
    return df
def stock_buy_sell(df, target_price_per, stop_loss_per, holding_period):
    df['Buy'] = 0
    for i in range(len(df) - holding_period):
        current_price = df[('Open', 'AMD')].iloc[i]
        for j in range(1,holding_period+1):
            if i+j>=len(df):
                break
            max_price_gain = df[('High', 'AMD')].iloc[i+j] - current_price
            max_price_lose = current_price - df[('Low', 'AMD')].iloc[i+j]
            if (max_price_gain >= current_price * target_price_per) and (max_price_lose <= current_price * stop_loss_per):
                df.at[df.index[i], 'Buy'] = 1
                break
    return df
    

df = download_stock_data('AMD').reset_index()
print(df.columns)
df = stock_add_indicator(df,5,200)
df_cleaned = df.dropna()
df_cleaned = stock_buy_sell(df_cleaned,0.003,0.001,5)
# Set display options to show more rows and columns
pd.set_option('display.max_rows', None)  # None will display all rows
pd.set_option('display.max_columns', None)  # None will display all columns
pd.set_option('display.width', None)  # Remove line width restrictions
pd.set_option('display.max_colwidth', None)  # Show all content in each column

print(df_cleaned.head(5))
y = df_cleaned['Buy']
X = df_cleaned.iloc[:,:-1]
x_train, x_test, y_train, y_test = train_test_split(X,y,train_size=0.8, test_size=0.2, random_state=1)
lr = LogisticRegression().fit(X,y)
y_pred = lr.predict(x_test)
print(accuracy_score(y_test, y_pred))

[*********************100%***********************]  1 of 1 completed
/var/folders/yz/7szystj11p33mq6mw_djrn8h0000gn/T/ipykernel_43170/1389699711.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Buy'] = 0


MultiIndex([(  'Date',    ''),
            ( 'Close', 'AMD'),
            (  'High', 'AMD'),
            (   'Low', 'AMD'),
            (  'Open', 'AMD'),
            ('Volume', 'AMD')],
           names=['Price', 'Ticker'])
Price        Date      Close       High        Low       Open    Volume  \
Ticker                   AMD        AMD        AMD        AMD       AMD   
199    2020-10-15  83.129997  84.720001  82.419998  83.400002  33696400   
200    2020-10-16  83.169998  83.989998  82.410004  83.540001  31474300   
201    2020-10-19  82.000000  84.650002  81.529999  83.620003  36689100   
202    2020-10-20  81.559998  82.690002  80.580002  82.160004  38219400   
203    2020-10-21  79.199997  81.930000  79.190002  81.720001  36426400   

Price         SMA       LMA Buy  
Ticker                           
199     84.001999  60.04705   0  
200     84.015999  60.21740   0  
201     83.557999  60.38440   0  
202     82.813998  60.55025   0  
203     81.811998  60.70500   0  


DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>)